# 单类型 lennard_jones MD

## Import part

In [ ]:
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
from matplotlib import style
plt.style.use('seaborn-v0_8-whitegrid')
import numpy as np
from PIL import Image
import glob
import moviepy.editor as mp
from datetime import datetime
import time
import os.path
from os import path
import shutil
from IPython.display import display
import random

In [ ]:
#输入文件(可自定义):
#dt = 0.005      #时间步长
#density =0.8       #流体密度
#initUcell_x = 20   #方格的x大小
#initUcell_y = 20   #y大小
#initUcell_z = 20
#stepAvg = 100       #用来抽样统计步数
#stepEquil = 0     #平衡步数
#num_steps = 500   #模拟循环数
#T = 1.   #温度设置

## Definition of mol and prop

In [ ]:
class Mol():
    def __init__(self,pos,vel,a):
        """
        每一分子对象（mol）所具有的属性如下：
        pos:原子坐标
        vel:原子速度
        a:原子加速度向量
        """
        #初始化以上属性的值
        self.pos=np.asarray([0.0,0.0,0.0])
        self.vel=np.asarray([0.0,0.0,0.0])
        self.a=np.asarray([0.0,0.0,0.0])
        
# 体系属性类，主要用来求体系属性（测定量）的平均值和标准差        
class Prop():
   def __init__(self, val, sum, sum_squ ):
        self.val=val                       #体系测定量的值
        self.sum=sum                       #体系测定量的值的和
        self.sum_squ=sum_squ               #体系测定量的值的平方和

## Toolfuntions

In [ ]:
# 返回平方和立方值的函数：
def Sqr(x):
    return (x * x)


def Cube(x):
    return ((x) * (x) * (x))



# 用于生成 (0,1) 范围内的均匀分布的随机数
##def RandR():
    global randSeedP
    randSeedP=(randSeedP * IMUL + IADD) & MASK
    return (randSeedP*SCALE)## ?

# 用于在三维空间产生具有均匀分布的随机方向的单位向量
def VRand(p):
    s: float
    phi: float
    s = 2. * math.pi * random.uniform(0, 1)
    phi = 2. * math.pi * random.uniform(0, 1)
    p[0] = math.sin(s)*math.cos(phi)      # x-component of vector p
    p[1] = math.sin(s)*math.sin(phi)      # y-component of vector p
    p[2] = math.cos(s)                    # z-component of vector p
    return p



# 周期性边界（PBC）算法：
"""
第一部分如下：
体系内粒子的坐标的约束：
  三维体系中，一个方体元胞四周有相同的元胞(称为"镜像")，
  若一个粒子从右边边界穿出，操作上左边边界对应位置有相同粒子进入，
  由此保证了元胞中粒子数目即物理量如动量与能量的守恒。
  因此有以下结论：
  region[0]为x方向的元胞的边长，则粒子在x方向的坐标区间为：[-0.5*region[0],0.5*region[0]]
  regino[1]为y方向的元胞的边长, 则粒子在x方向的坐标区间为：[-0.5*region[1],0.5*region[1]]
  regino[2]为z方向的元胞的边长, 则粒子在x方向的坐标区间为：[-0.5*region[2],0.5*region[2]]

  以x方向元胞为例：假设边长为1x1x1的正方体元胞 
  当粒子坐标为x>=0.5时，为了确保当前粒子在一个1x1元胞中，有关系：x-新位置=1，
  故有：新位置=x-1
  同理得： x<-0.5时，有关系：新位置-（x）=1  故有:新位置=1+x，
  
第二部分：对粒子间距离计算使用最小镜像法则，见：ComputerForces()函数
"""
#PBC算法第一部分：对体系内粒子坐标的约束
def VWrapAll(v):
    #对元胞x方向的坐标约束
    if v[0]>=0.5*region[0]:    
        v[0]-=region[0]
    elif v[0]<-0.5*region[0]:
         v[0] +=region[0]
    
    #对元胞y方向的坐标约束
    if v[1] >= 0.5 * region[1]:
        v[1] -= region[1]
    elif v[1] < -0.5 * region[1]:
        v[1] += region[1]

    #对元胞z方向的坐标约束
    if v[0]>=0.5*region[2]:    
        v[0]-=region[2]
    elif v[0]<-0.5*region[2]:
         v[0] +=region[2]

## Main program

In [ ]:
mov = 1                                      # 如果你想做一个视频的话，设置 mov=1
workdir = str(os.getcwd()+'/')              # 为所有png和视频设置一个工作目录

# 如果/LJP目录不存在，就建立它，否则就删除/LJP（和它的内容）
# 创建一个新的/LJP目录。
if path.exists(str(workdir+'LJP'))==False:
    os.makedirs(str(workdir+'LJP'))
else:
    shutil.rmtree(str(workdir+'LJP'))
    os.makedirs(str(workdir+'LJP'))

# 加载输入参数文件
df_params = pd.read_csv('LJP.csv', sep=',', header=None, names=['parameter', 'value'])

#初始该模拟系统热力学属性值
NDIM = 3                        #维度设置
vSum = np.asarray([0.0, 0.0, 0.0])   #速度之和
kinEnergy =Prop(0.0, 0.0, 0.0)  #动能
totEnergy =Prop(0.0, 0.0, 0.0)  #势能
pressure  =Prop(0.0, 0.0, 0.0)  #压强

systemParams = []              #初始化系统参数列表

#用于产生随机数算法函数的变量
IADD = 453806245
IMUL = 314159269
MASK = 2147483647
SCALE = 0.4656612873e-9
randSeedP = 17

#初始化参数：把LJP.csv文件中的参数值传递给MD模拟所需要的相关变量
dt = float(df_params.values[0][1])              #模拟步长
density = float(df_params.values[1][1])             #体系密度


initUcell = np.asarray([0.0, 0.0, 0.0])            #初始化元胞，具体大小由输入文件密度决定
initUcell[0] = int(df_params.values[2][1])         #元胞X方向长度=20
initUcell[1] = int(df_params.values[3][1])         #元胞y方向长度=20
initUcell[2] = int(df_params.values[4][1])         #元胞z方向长度=20
 
stepAvg = int(df_params.values[5][1])              #抽样平均数
stepEquil = float(df_params.values[6][1])          #平衡步长
stepLimit = float(df_params.values[7][1])          #限制的步数
T = float(df_params.values[8][1])        #体系温度

#定义了一个Mol类对象的数组，大小为20*20*20=8000. mol[0]~mol[7999]，共7999个氩原子分子
mol = [Mol(np.asarray([0.0, 0.0, 0.0]), \
           np.asarray([0.0, 0.0, 0.0]), \
           np.asarray([0.0, 0.0, 0.0])) for i in range(int(initUcell[0]*initUcell[1]*initUcell[2]))]

global nMol        #定义分子的数量变量
nMol = len(mol)    #为mol数组的长度为8000

# 氩-氩相互作用的LJ势参数:
epsilon =  1
sigma = 1

# 执行体系初始化相关功能函数
SetParams()                                   #设置主程序内部模拟所需参数
SetupJob()                                    #初始化粒子的坐标，速度，加速
moreCycles = 1                                #MD循环控制开关变量1：run; 0:stop

n = 0                                        #记录步数，用来给每一步输出的坐标图命名                
while moreCycles:                            #MD循环
    SingleStep()                             #求解每一步的运动方程
    if mov==1:
        plotMolLjp(mol, workdir, n)          #制作单个步长的粒子坐标图
    n += 1
    if stepCount >= stepLimit:
        moreCycles = 0
        
#输出模拟过程中生成的参数
columns = ['timestep','timeNow', '$\Sigma v$', 'E', '$\sigma E$', 'Ek', '$\sigma Ek$', 'P_1', 'P_2']
df_systemParams = pd.DataFrame(systemParams, columns=columns)   

#绘制表格，仅在jupyter notebook使用，打印出数据列表
display(df_systemParams) 

if mov==1:                     
    makeMov()                 # 制作视频
GraphOutput()                 #输出体系测定量随模拟时间变化的图像

## Functions related to initilation

In [ ]:
"""
模拟所需的其他变量，不包括那些构成输入数据的变量，
由函数SetParams设置。
"""
def SetParams():
    global rCut    #截断距离
    global region  #区域
    global velMag  #速度变化幅度
    
    #截断值取势阱底部时的原子间距离,即rmin=rCut=2^1/6 *sigma
    rCut=math.pow(2.,1./6.) * sigma
    
    #计算在预设密度条件下，粒子所在区域的大小
    """
    三维空间下的：
    desity=粒子数目/体积，
    平衡体系模拟，密度保持不变，粒子数目不变则需要根据密度来更新粒子活动区域的大小，如下:
    """
    region=np.multiply(1./math.pow(density,1./3.),initUcell) #？
    nMol=len(mol)
    
    #速度的变化幅度取决于温度的高低
    velMag = math.sqrt(NDIM * (1. -1. /nMol) * T)

    """
所有初始化计算的工作都集中在以下的函数。
"""
def SetupJob(): 
    global stepCount   #步长计数
    
    stepCount = 0  #初始化步长计数变量的值
    InitCoords()   #初始化坐标
    InitVels()     #初始化速度
    InitAccels()   #初始化加速速度
    AccumProps(0)  #初始化系统属性值（总能量，动能，压强）

"""
代码7 ：InitCoords()，InitVels()，InitAccels()

这里主要是初始化体系粒子的坐标，速度，加速度的函数
"""

# 初始化坐标。
# 这里使用了一个简单的正方形晶格（可以选择不等长的边）,
# 因此，每个元胞只包含一个原子，系统以原点为中心，
def InitCoords():
    c=np.asarray([0.0,0.0,0.0])                       #初始坐标值
    gap=np.divide(region,initUcell)                   #单个粒子所在元胞的大小
    n=0                                               #给粒子计数
    
    #将8000个原子分别放到元胞中（这些元胞向x,y,z方向扩胞成了region区域）
    for nz in range(0, int(initUcell[2])):
        for ny in range(0, int(initUcell[1])):
            for nx in range(0, int(initUcell[0])):
                c = np.asarray([nx+0.5, ny+0.5, nz+0.5])
                c = np.multiply(c, gap)
                c = np.add(c, np.multiply(-0.5, region))
                mol[n].pos = c
                n=n+1

            
# 初始化速度。
# 初始速度被设置为固定幅度（velMag）,
# 这取决于温度。在分配了随机速度方向后
# 调整速度方向，以确保质心是静止的，因为系统在不受净合外力作用下，其质心速度应该保持不变,
# 函数vRand作为均匀分布的径向单位向量的来源。
def InitVels():
    global vSum
    vSum=np.zeros(vSum.shape)   #返回特定大小，以0填充新的数组:[0,0]，这里是形状
    
    for n in range(nMol):
        VRand(mol[n].vel)                               #产生随机随机速度向量[x,y,z]
        mol[n].rv=np.multiply(mol[n].vel,velMag)        #根据温度，生成新的速度
        vSum=np.add(vSum,mol[n].vel)                    #质心的速度，系统总速度各质点速度的总和
    
    #调整度方向，以确保质心是静止的,这里取8000个粒子在x,y,z方向的平均速度1/8000*Vsum+每一个原子的速度
    for n in range (nMol):
        mol[n].vel=np.add(mol[n].vel,np.multiply((- 1.0 / nMol),vSum))

        
#初始化加速度。
# 加速度被初始化为零[0,0]
def InitAccels():
    for n in range(nMol):
        mol[n].a=np.zeros(mol[n].a.shape)

#初始化值
def PropZero(v):
    v.sum = v.sum_squ = 0.
    return v    
#求和与平方和    
def PropAccum(v):
    v.sum += v.val
    v.sum_squ += Sqr(v.val)
    return v    

#求平均值和标准差
def PropAvg(v, n):
    v.sum /= n
    v.sum_squ = math.sqrt(max(v.sum_squ / n - Sqr(v.sum), 0.)) 
    return v 


# AccumProps：收集测定量的结果，并根据要求计算平均值和标准偏差。
def AccumProps(icode):
    
    if icode == 0:             # 0：初始化                                            
        PropZero(totEnergy)
        PropZero(kinEnergy)
        PropZero(pressure) 
    if icode == 1:            # 1：求和                    
        PropAccum(totEnergy)
        PropAccum(kinEnergy)
        PropAccum(pressure)    
    if icode == 2:           # 2：求平均值和标准差
        PropAvg(totEnergy, stepAvg)
        PropAvg(kinEnergy, stepAvg)
        PropAvg(pressure, stepAvg) 


## Simulation

### Singlestep

In [ ]:
def SingleStep():
    
    global stepCount # 步长计数
    global timeNow   #  模拟运行时间

    stepCount +=1                   
    timeNow = stepCount * dt   #模拟运行的时间=步数x步长
    LeapfrogStep(1)                #求解运动方程积分
    ApplyBoundaryCond()           #应用周期性边界条件
    ComputeForces()              # 计算原间相互作用力
    LeapfrogStep(2)             # 坐标和速度的积分
    EvalProps()                #计算系统属性值（速度，速度平方和，总能量，动能，压力）
    AccumProps(1)             #系统属性值求和
    
    #每一百步统计系统的属性值（0,100,200,300,400,500），可以设置stepAvg的值进行自定义
    if (stepCount % stepAvg == 0):
        AccumProps(2)                         #求系统的属性值的平均值和标准差
        systemParams.append(PrintSummary())   #将结果加入到 systemParams列表中
        AccumProps(0)                         # 重置系统属性值用来下一次的统计

### leapfrogstep

In [ ]:
"""
所谓积分方法就是根据粒子的位置，速度和受力计算deltT时刻之后的新位置和新速度算法。
运动方程的积分使用一种简单的数值技术：跃迁法。该方法具有良好的能量守恒特性，
LeapfrogStep整合了坐标和速度。其参数部分决定了两步跃迁过程的哪一部分需要执行。
vix(t + h/2) = vix(t) + (h/2)aix(t)
rix(t + h) = rix(t) + hvix (t + h/2)
"""

def LeapfrogStep(part):
    if part == 1:
        for n in range (nMol):
            mol[n].vel=np.add(mol[n].vel,np.multiply(0.5 * dt,mol[n].a))
            mol[n].pos=np.add(mol[n].pos,np.multiply(dt,mol[n].vel))
    else :
        for n in range(nMol):
            mol[n].vel=np.add(mol[n].vel,np.multiply(0.5 * dt,mol[n].a))


### ApplyBoundaryCond

In [ ]:
# 应用周期性边界，约束粒子坐标位置，保持元胞粒子数恒定 
def ApplyBoundaryCond():
    for n in range(nMol):
        VWrapAll(mol[n].pos)

### ComputerForces

In [ ]:
"""
#计算原子间相互作用力函数
ComputeForce函数数：负责根据粒子（原子）的位置计算计算每一个粒子的受力，
由受力情况进而求得加速度。

该函数实现了LJ_force，并计算了位于ri和rj的每一对原子i和j的加速度和力。

rCut=极限分离临界值（rc），它是：rCut=math.pow(2., 1./6.)
当r向rCut增加时，力下降到0。

牛顿第三定律意味着fji=-fij，所以每个原子对只需要检查一次。
"""

def ComputeForces():
    global virSum                    #用于计算压强的中间变量（fij*rij）和
    global uSum                      #LJ势能和
    fcVal=0                          # 原子j对原子i施加的力
    rrCut=Sqr(rCut)                  # rCut:Rc,截断半径的平方
    
    #初始化分子加速度
    for n in range (nMol):
        mol[n].a=np.zeros(mol[n].a.shape)
        
    uSum=0.                        #初始化LJ势能和值
    virSum=0.
    n=0
    for j1 in range(nMol-1):
        for j2 in range(j1+1,nMol):
            
            # 使Delta Rij。(RJ1-RJ2的平方之和)
            dr=np.subtract(mol[j1].pos,mol[j2].pos) # dr包含Rj1和Rj2之间的x,y坐标差值
            VWrapAll(dr)                        #应用PBC约束原子在元胞内,更新了dr[0],dr[1]
            rr=(dr[0] * dr[0] + dr[1] * dr[1]+ dr[2]*dr[2]) #两原子距离的平方，这里并未求两原子间距离的绝对值，没必要，因为与截断半径只是比大小，省去了平方根的计算
            r=np.sqrt(rr)                      #r两为原子间的距离     
            
            # 这里是使用原子距离的平方来 dr2 < Rc^2 来判断两原子键相互作用力，
            if(rr < rrCut):
                """
                在原文中：作者认为LJ势尾部的吸引项对该体系模型特征不敏感，所以为了简化计算去掉了
                所以原文有以下代码：
                rri = sigma / rr                 
                rri3 = Cube(rri)
                fcVal = 48. * rri3 * (rri3 - 0.5) * rri
                """
                #本例实现了完整的LJ势能计算，加入了尾部吸引项的描述，所以后续的测定量的计算结果会与原文存在差异
                fcVal = 48 * epsilon * np.power(sigma, 12) / np.power(r, 13) - 24 * epsilon * np.power(sigma, 6) / np.power(r, 7)
               
                # 更新加速度
                mol[j1].a = np.add(mol[j1].a, np.multiply(fcVal, dr))
                mol[j2].a = np.add(mol[j2].a, np.multiply(-fcVal, dr))
                
                #LJ势能计算
                uSum += 4 * epsilon * np.power(sigma/r, 12)/r - np.power(sigma/r, 6)    
                virSum += fcVal * rr

### EvalProps

In [ ]:
# 计算体系热力学属性值（测定值）
def EvalProps():
    global vSum
    vvSum = 0.                    #系统速度平方的总和
    vSum = np.zeros(vSum.shape)   #质心速度，为系统速度=各质点速度总和，初始化为[0,0]
    
    global kinEnergy                 #动能
    global totEenergy                #总能量
    global pressure                  #压强
    
    #求得质心速度
    for n in range(nMol):
        vSum =np.add(vSum,mol[n].vel)                                        
        vv = (mol[n].vel[0] * mol[n].vel[0] + mol[n].vel[1] * mol[n].vel[1] + mol[n].vel[2] * mol[n].vel[2])   #xv^2+yv^2+zv^2
        vvSum += vv
        
        # 在三维分子体系中，热力学属性值的数值计算方法
        kinEnergy.val = (0.5 * vvSum) / nMol                         #单个原子动能，nMol代表原子数
        totEnergy.val = kinEnergy.val + (uSum / nMol)                #总能量：单个原子的动能+单个原子的势能
        pressure.val = density * (vvSum + virSum) / (nMol * NDIM)    #体系压强
        

### PrintSummary

In [ ]:
# 打印输出体系的测定量
def PrintSummary():
    #打印测定量，保留小数点后四位
    print(stepCount, \
          "{0:.4f}".format(timeNow), \
          "{0:.4f}".format(vSum[0] / nMol) ,\
          "{0:.4f}".format(totEnergy.sum),\
          "{0:.4f}".format(totEnergy.sum_squ), \
          "{0:.4f}".format(kinEnergy.sum), \
          "{0:.4f}".format(kinEnergy.sum_squ),\
          "{0:.4f}".format(pressure.sum),\
          "{0:.4f}".format(pressure.sum_squ))
    
    #返回精确的测定量值
    return (stepCount, \
          timeNow, \
          (vSum[0] / nMol) ,\
          totEnergy.sum,\
          totEnergy.sum_squ, \
          kinEnergy.sum, \
          kinEnergy.sum_squ,\
          pressure.sum,\
          pressure.sum_squ)    

## Postprocessing part

### plotMolLjp 

In [ ]:
def plotMolLjp(mol,workdir,n):
    import matplotlib.patches as mpatches
    import matplotlib.pyplot as plt
    
    Time=timeNow                                 #模拟时长
    Sigma_v = "{0:.4f}".format(vSum[0] / nMol)
    E = "{0:.4f}".format(totEnergy.sum)
    Sigma_E = "{0:.4f}".format(totEnergy.sum_squ)
    Ek = "{0:.4f}".format(kinEnergy.sum)
    Sigma_Ek = "{0:.4f}".format(kinEnergy.sum_squ)
    P_1 = "{0:.4f}".format(pressure.sum)
    P_2 = "{0:.4f}".format(pressure.sum_squ)  
    
    #该代码能使图表在jupyter notebook中直接显示，在.py代码中不需要
    %matplotlib inline  
    
    TileName = (workdir+'LJP/'+str(n)+'.png')      #传入的n表示n步，这里用来给生成的图像命名
    x = []                                         #新建列表保存粒子x的坐标
    y = []                                         #新建列表保存粒子y的坐标
    z = []                                         #新建列表保存粒子z的坐标

    # 遍历0-7999号原子的坐标，加入到列表中去
    for n in range(len(mol)):
        x.append(mol[n].pos[0])
        y.append(mol[n].pos[1])
        z.append(mol[n].pos[2])

    # 标记400个原子中两个位置较为居中的两相邻原子mol[250]和mol[251],用于观察
    mark_1 = int(len(mol)/2 + len(mol)/8)
    mark_2 = int(len(mol)/2 + len(mol)/8 + 1)
    
    # 根据原子坐标绘制图形
    plt.plot(x, y, z, 'o', color='blue')                   #每个原子均为蓝色圆形
    plt.plot(x[mark_1], y[mark_1],z[mark_1], 'o', color='red')    #标记mark_1为红色
    plt.plot(x[mark_2], y[mark_2],z[mark_2], 'o', color='cyan')   #标记mark_2为青色
    
    # 绘制图标的标题
    plt.title('timestep:'+"{0:.4f}".format(timeNow)+'; '+\
              '$\Sigma v$:'+Sigma_v+'; '+\
              'E:'+E+'; '+\
              '$\sigma E$:'+Sigma_E+';\n'+\
              'Ek:'+Ek+'; ' +\
              '$\sigma Ek$:'+Sigma_Ek+'; '+\
              'P.sum1:'+P_1+'; '+\
              'P.sum2:'+P_2+'; ', loc='left')
    
   
    plt.savefig(TileName, dpi=100)                       #保存为坐标图为.png图片

### makeMov

In [ ]:
# 制作粒子运动轨迹视频
def makeMov():
    
    # 将多张.png图像转为gif图的片段
    frames = []
    imgs = sorted(glob.glob('LJP/*.png'), key=os.path.getmtime)
    for i in imgs:
        temp = Image.open(i)
        keep = temp.copy()
        frames.append(keep)
        temp.close()
        
    # 删除全部的.png图
    for i in imgs:
        os.remove(i)        

    # 片段合并保存为GIF文件，永远循环播放
    frames[0].save('LJP/coordinates.gif', format='GIF',
                   append_images=frames[1:],
                   save_all=True,
                   duration=30, loop=0)

    # 将GIF图转换成MP4视频文件
    clip = mp.VideoFileClip("LJP/coordinates.gif")
    clip.write_videofile("LJP/"+"coordinates"+".mp4")
    
    # 删除gif图
    os.remove("LJP/coordinates.gif")

### GraphOutput

In [ ]:
#绘制和系统属性之相关的图像
def GraphOutput():
   
    ax = \
    df_systemParams.plot(x="timestep", y='$\Sigma v$', kind="line")
    df_systemParams.plot(x="timestep", y='E', kind="line", ax=ax, color="C1")
    df_systemParams.plot(x="timestep", y='$\sigma E$', kind="line", ax=ax, color="C2")
    df_systemParams.plot(x="timestep",  y='Ek', kind="line", ax=ax, color="C3")
    df_systemParams.plot(x="timestep", y='$\sigma Ek$', kind="line", ax=ax, color="C4")
    df_systemParams.plot(x="timestep", y='P_1', kind="line", ax=ax, color="C9")
    df_systemParams.plot(x="timestep", y='P_2', kind="line", ax=ax, color="C9")
    
    plt.savefig('plot.jpg', dpi=300)

# 完整代码

In [ ]:
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')
import numpy as np
from PIL import Image
import glob
import moviepy.editor as mp
from datetime import datetime
import time
import os.path
from os import path
import shutil
from IPython.display import display
import random
class Mol():
    def __init__(self,pos,vel,a):
        #初始化以上属性的值
        self.pos=np.asarray([0.0,0.0,0.0])
        self.vel=np.asarray([0.0,0.0,0.0])
        self.a=np.asarray([0.0,0.0,0.0])
        
# 体系属性类，主要用来求体系属性（测定量）的平均值和标准差        
class Prop():
   def __init__(self, val, sum, sum_squ ):
        self.val=val                       #体系测定量的值
        self.sum=sum                       #体系测定量的值的和
        self.sum_squ=sum_squ               #体系测定量的值的平方和
    # 返回平方和立方值的函数：
def Sqr(x):
    return (x * x)


def Cube(x):
    return ((x) * (x) * (x))



# 用于生成 (0,1) 范围内的均匀分布的随机数
##def RandR():
    global randSeedP
    randSeedP=(randSeedP * IMUL + IADD) & MASK
    return (randSeedP*SCALE)## ?

# 用于在三维空间产生具有均匀分布的随机方向的单位向量
def VRand(p):
    s: float
    phi: float
    s = 2. * math.pi * random.uniform(0, 1)
    phi = 2. * math.pi * random.uniform(0, 1)
    p[0] = math.sin(s)*math.cos(phi)      # x-component of vector p
    p[1] = math.sin(s)*math.sin(phi)      # y-component of vector p
    p[2] = math.cos(s)                    # z-component of vector p
    return p



# 周期性边界（PBC）算法：

#PBC算法第一部分：对体系内粒子坐标的约束
def VWrapAll(v):
    #对元胞x方向的坐标约束
    if v[0]>=0.5*region[0]:    
        v[0]-=region[0]
    elif v[0]<-0.5*region[0]:
         v[0] +=region[0]
    
    #对元胞y方向的坐标约束
    if v[1] >= 0.5 * region[1]:
        v[1] -= region[1]
    elif v[1] < -0.5 * region[1]:
        v[1] += region[1]

    #对元胞z方向的坐标约束
    if v[0]>=0.5*region[2]:    
        v[0]-=region[2]
    elif v[0]<-0.5*region[2]:
         v[0] +=region[2]


def SetParams():
    global rCut    #截断距离
    global region  #区域
    global velMag  #速度变化幅度
    
    #截断值取势阱底部时的原子间距离,即rmin=rCut=2^1/6 *sigma
    rCut=math.pow(2.,1./6.) * sigma
    
    #计算在预设密度条件下，粒子所在区域的大小

    region=np.multiply(1./math.pow(density,1./3.),initUcell) #？
    nMol=len(mol)
    
    #速度的变化幅度取决于温度的高低
    velMag = math.sqrt(NDIM * (1. -1. /nMol) * T)



# 初始化坐标。
# 这里使用了一个简单的正方形晶格（可以选择不等长的边）,
# 因此，每个元胞只包含一个原子，系统以原点为中心，
def InitCoords():
    c=np.asarray([0.0,0.0,0.0])                       #初始坐标值
    gap=np.divide(region,initUcell)                   #单个粒子所在元胞的大小
    n=0                                               #给粒子计数
    
    #将8000个原子分别放到元胞中（这些元胞向x,y,z方向扩胞成了region区域）
    for nz in range(0, int(initUcell[2])):
        for ny in range(0, int(initUcell[1])):
            for nx in range(0, int(initUcell[0])):
                c = np.asarray([nx+0.5, ny+0.5, nz+0.5])
                c = np.multiply(c, gap)
                c = np.add(c, np.multiply(-0.5, region))
                mol[n].pos = c
                n=n+1

            
# 初始化速度。
# 初始速度被设置为固定幅度（velMag）,
# 这取决于温度。在分配了随机速度方向后
# 调整速度方向，以确保质心是静止的，因为系统在不受净合外力作用下，其质心速度应该保持不变,
# 函数vRand作为均匀分布的径向单位向量的来源。
def InitVels():
    global vSum
    vSum=np.zeros(vSum.shape)   #返回特定大小，以0填充新的数组:[0,0]，这里是形状
    
    for n in range(nMol):
        VRand(mol[n].vel)                               #产生随机随机速度向量[x,y,z]
        mol[n].vel=np.multiply(mol[n].vel,velMag)        #根据温度，生成新的速度
        vSum=np.add(vSum,mol[n].vel)                    #质心的速度，系统总速度各质点速度的总和
    
    #调整度方向，以确保质心是静止的,这里取8000个粒子在x,y,z方向的平均速度1/8000*Vsum+每一个原子的速度
    for n in range (nMol):
        mol[n].vel=np.add(mol[n].vel,np.multiply((- 1.0 / nMol),vSum))

        
#初始化加速度。
# 加速度被初始化为零[0,0]
def InitAccels():
    for n in range(nMol):
        mol[n].a=np.zeros(mol[n].a.shape)

#初始化值
def PropZero(v):
    v.sum = v.sum_squ = 0.
    return v    
#求和与平方和    
def PropAccum(v):
    v.sum += v.val
    v.sum_squ += Sqr(v.val)
    return v    

#求平均值和标准差
def PropAvg(v, n):
    v.sum /= n
    v.sum_squ = math.sqrt(max(v.sum_squ / n - Sqr(v.sum), 0.)) 
    return v 


# AccumProps：收集测定量的结果，并根据要求计算平均值和标准偏差。
def AccumProps(icode):
    
    if icode == 0:             # 0：初始化                                            
        PropZero(totEnergy)
        PropZero(kinEnergy)
        PropZero(pressure) 
    if icode == 1:            # 1：求和                    
        PropAccum(totEnergy)
        PropAccum(kinEnergy)
        PropAccum(pressure)    
    if icode == 2:           # 2：求平均值和标准差
        PropAvg(totEnergy, stepAvg)
        PropAvg(kinEnergy, stepAvg)
        PropAvg(pressure, stepAvg) 


def SetupJob(): 
    global stepCount   #步长计数
    
    stepCount = 0  #初始化步长计数变量的值
    InitCoords()   #初始化坐标
    InitVels()     #初始化速度
    InitAccels()   #初始化加速速度
    AccumProps(0)  #初始化系统属性值（总能量，动能，压强）



def LeapfrogStep(part):
    if part == 1:
        for n in range (nMol):
            mol[n].vel=np.add(mol[n].vel,np.multiply(0.5 * dt,mol[n].a))
            mol[n].pos=np.add(mol[n].pos,np.multiply(dt,mol[n].vel))
    else :
        for n in range(nMol):
            mol[n].vel=np.add(mol[n].vel,np.multiply(0.5 * dt,mol[n].a))

# 应用周期性边界，约束粒子坐标位置，保持元胞粒子数恒定 
def ApplyBoundaryCond():
    for n in range(nMol):
        VWrapAll(mol[n].pos)

def ComputeForces():
    global virSum                    #用于计算压强的中间变量（fij*rij）和
    global uSum                      #LJ势能和
    fcVal=0                          # 原子j对原子i施加的力
    rrCut=Sqr(rCut)                  # rCut:Rc,截断半径的平方
    
    #初始化分子加速度
    for n in range (nMol):
        mol[n].a=np.zeros(mol[n].a.shape)
        
    uSum=0.                        #初始化LJ势能和值
    virSum=0.
    n=0
    for j1 in range(nMol-1):
        for j2 in range(j1+1,nMol):
            
            # 使Delta Rij。(RJ1-RJ2的平方之和)
            dr=np.subtract(mol[j1].pos,mol[j2].pos) # dr包含Rj1和Rj2之间的x,y坐标差值
            VWrapAll(dr)                        #应用PBC约束原子在元胞内,更新了dr[0],dr[1]
            rr=(dr[0] * dr[0] + dr[1] * dr[1]+ dr[2]*dr[2]) #两原子距离的平方，这里并未求两原子间距离的绝对值，没必要，因为与截断半径只是比大小，省去了平方根的计算
            r=np.sqrt(rr)                      #r两为原子间的距离     
            
            # 这里是使用原子距离的平方来 dr2 < Rc^2 来判断两原子键相互作用力，
            if(rr < rrCut):
           
                #本例实现了完整的LJ势能计算，加入了尾部吸引项的描述，所以后续的测定量的计算结果会与原文存在差异
                fcVal = 48 * epsilon * np.power(sigma, 12) / np.power(r, 13) - 24 * epsilon * np.power(sigma, 6) / np.power(r, 7)
               
                # 更新加速度
                mol[j1].a = np.add(mol[j1].a, np.multiply(fcVal, dr))
                mol[j2].a = np.add(mol[j2].a, np.multiply(-fcVal, dr))
                
                #LJ势能计算
                uSum += 4 * epsilon * np.power(sigma/r, 12)/r - np.power(sigma/r, 6)    
                virSum += fcVal * rr

# 计算体系热力学属性值（测定值）
def EvalProps():
    global vSum
    vvSum = 0.                    #系统速度平方的总和
    vSum = np.zeros(vSum.shape)   #质心速度，为系统速度=各质点速度总和，初始化为[0,0]
    
    global kinEnergy                 #动能
    global totEnergy                #总能量
    global pressure                  #压强
    
    #求得质心速度
    for n in range(nMol):
        vSum =np.add(vSum,mol[n].vel)                                        
        vv = (mol[n].vel[0] * mol[n].vel[0] + mol[n].vel[1] * mol[n].vel[1] + mol[n].vel[2] * mol[n].vel[2])   #xv^2+yv^2+zv^2
        vvSum += vv
        
        # 在三维分子体系中，热力学属性值的数值计算方法
        kinEnergy.val = (0.5 * vvSum) / nMol                         #单个原子动能，nMol代表原子数
        totEnergy.val = kinEnergy.val + (uSum / nMol)                #总能量：单个原子的动能+单个原子的势能
        pressure.val = density * (vvSum + virSum) / (nMol * NDIM)    #体系压强

# 打印输出体系的测定量
def PrintSummary():
    #打印测定量，保留小数点后四位
    print(stepCount, \
          "{0:.4f}".format(timeNow), \
          "{0:.4f}".format(vSum[0] / nMol) ,\
          "{0:.4f}".format(totEnergy.sum),\
          "{0:.4f}".format(totEnergy.sum_squ), \
          "{0:.4f}".format(kinEnergy.sum), \
          "{0:.4f}".format(kinEnergy.sum_squ),\
          "{0:.4f}".format(pressure.sum),\
          "{0:.4f}".format(pressure.sum_squ))
    
    #返回精确的测定量值
    return (stepCount, \
          timeNow, \
          (vSum[0] / nMol) ,\
          totEnergy.sum,\
          totEnergy.sum_squ, \
          kinEnergy.sum, \
          kinEnergy.sum_squ,\
          pressure.sum,\
          pressure.sum_squ)    

def SingleStep():
    
    global stepCount # 步长计数
    global timeNow   #  模拟运行时间

    stepCount +=1                   
    timeNow = stepCount * dt   #模拟运行的时间=步数x步长
    LeapfrogStep(1)                #求解运动方程积分
    ApplyBoundaryCond()           #应用周期性边界条件
    ComputeForces()              # 计算原间相互作用力
    LeapfrogStep(2)             # 坐标和速度的积分
    EvalProps()                #计算系统属性值（速度，速度平方和，总能量，动能，压力）
    AccumProps(1)             #系统属性值求和
    
    #每一百步统计系统的属性值（0,100,200,300,400,500），可以设置stepAvg的值进行自定义
    if (stepCount % stepAvg == 0):
        AccumProps(2)                         #求系统的属性值的平均值和标准差
        systemParams.append(PrintSummary())   #将结果加入到 systemParams列表中
        AccumProps(0)                         # 重置系统属性值用来下一次的统计

def plotMolLjp(mol,workdir,n):
    import matplotlib.patches as mpatches
    import matplotlib.pyplot as plt
    
    Time=timeNow                                 #模拟时长
    Sigma_v = "{0:.4f}".format(vSum[0] / nMol)
    E = "{0:.4f}".format(totEnergy.sum)
    Sigma_E = "{0:.4f}".format(totEnergy.sum_squ)
    Ek = "{0:.4f}".format(kinEnergy.sum)
    Sigma_Ek = "{0:.4f}".format(kinEnergy.sum_squ)
    P_1 = "{0:.4f}".format(pressure.sum)
    P_2 = "{0:.4f}".format(pressure.sum_squ)  
    
    #该代码能使图表在jupyter notebook中直接显示，在.py代码中不需要
    %matplotlib inline  
    
    TileName = (workdir+'LJP/'+str(n)+'.png')      #传入的n表示n步，这里用来给生成的图像命名
    x = []                                         #新建列表保存粒子x的坐标
    y = []                                         #新建列表保存粒子y的坐标
    z = []                                         #新建列表保存粒子z的坐标

    # 遍历0-7999号原子的坐标，加入到列表中去
    for n in range(len(mol)):
        x.append(mol[n].pos[0])
        y.append(mol[n].pos[1])
        z.append(mol[n].pos[2])

    # 标记400个原子中两个位置较为居中的两相邻原子mol[250]和mol[251],用于观察
    mark_1 = int(len(mol)/2 + len(mol)/8)
    mark_2 = int(len(mol)/2 + len(mol)/8 + 1)
    
    # 根据原子坐标绘制图形
    plt.plot(x, y, z, 'o', color='blue')                   #每个原子均为蓝色圆形
    plt.plot(x[mark_1], y[mark_1],z[mark_1], 'o', color='red')    #标记mark_1为红色
    plt.plot(x[mark_2], y[mark_2],z[mark_2], 'o', color='cyan')   #标记mark_2为青色
    
    # 绘制图标的标题
    plt.title('timestep:'+"{0:.4f}".format(timeNow)+'; '+\
              '$\Sigma v$:'+Sigma_v+'; '+\
              'E:'+E+'; '+\
              '$\sigma E$:'+Sigma_E+';\n'+\
              'Ek:'+Ek+'; ' +\
              '$\sigma Ek$:'+Sigma_Ek+'; '+\
              'P.sum1:'+P_1+'; '+\
              'P.sum2:'+P_2+'; ', loc='left')
    
   
    plt.savefig(TileName, dpi=100)                       #保存为坐标图为.png图片

# 制作粒子运动轨迹视频
def makeMov():
    
    # 将多张.png图像转为gif图的片段
    frames = []
    imgs = sorted(glob.glob('LJP/*.png'), key=os.path.getmtime)
    for i in imgs:
        temp = Image.open(i)
        keep = temp.copy()
        frames.append(keep)
        temp.close()
        
    # 删除全部的.png图
    for i in imgs:
        os.remove(i)        

    # 片段合并保存为GIF文件，永远循环播放
    frames[0].save('LJP/coordinates.gif', format='GIF',
                   append_images=frames[1:],
                   save_all=True,
                   duration=30, loop=0)

    # 将GIF图转换成MP4视频文件
    clip = mp.VideoFileClip("LJP/coordinates.gif")
    clip.write_videofile("LJP/"+"coordinates"+".mp4")
    
    # 删除gif图
    os.remove("LJP/coordinates.gif")

    #绘制和系统属性之相关的图像
def GraphOutput():
   
    ax = \
    df_systemParams.plot(x="timestep", y='$\Sigma v$', kind="line")
    df_systemParams.plot(x="timestep", y='E', kind="line", ax=ax, color="C1")
    df_systemParams.plot(x="timestep", y='$\sigma E$', kind="line", ax=ax, color="C2")
    df_systemParams.plot(x="timestep",  y='Ek', kind="line", ax=ax, color="C3")
    df_systemParams.plot(x="timestep", y='$\sigma Ek$', kind="line", ax=ax, color="C4")
    df_systemParams.plot(x="timestep", y='P_1', kind="line", ax=ax, color="C9")
    df_systemParams.plot(x="timestep", y='P_2', kind="line", ax=ax, color="C9")
    
    plt.savefig('plot.jpg', dpi=300)


mov = 1                                      # 如果你想做一个视频的话，设置 mov=1
workdir = str(os.getcwd()+'/')              # 为所有png和视频设置一个工作目录

# 如果/LJP目录不存在，就建立它，否则就删除/LJP（和它的内容）
# 创建一个新的/LJP目录。
if path.exists(str(workdir+'LJP'))==False:
    os.makedirs(str(workdir+'LJP'))
else:
    shutil.rmtree(str(workdir+'LJP'))
    os.makedirs(str(workdir+'LJP'))

# 加载输入参数文件
df_params = pd.read_csv('LJP.csv', sep=',', header=None, names=['parameter', 'value'])

#初始该模拟系统热力学属性值
NDIM = 3                        #维度设置
vSum = np.asarray([0.0, 0.0, 0.0])   #速度之和
kinEnergy =Prop(0.0, 0.0, 0.0)  #动能
totEnergy =Prop(0.0, 0.0, 0.0)  #势能
pressure  =Prop(0.0, 0.0, 0.0)  #压强

systemParams = []              #初始化系统参数列表

#用于产生随机数算法函数的变量
IADD = 453806245
IMUL = 314159269
MASK = 2147483647
SCALE = 0.4656612873e-9
randSeedP = 17

#初始化参数：把LJP.csv文件中的参数值传递给MD模拟所需要的相关变量
dt = float(df_params.values[0][1])              #模拟步长
density = float(df_params.values[1][1])             #体系密度


initUcell = np.asarray([0.0, 0.0, 0.0])            #初始化元胞，具体大小由输入文件密度决定
initUcell[0] = int(df_params.values[2][1])         #元胞X方向长度=20
initUcell[1] = int(df_params.values[3][1])         #元胞y方向长度=20
initUcell[2] = int(df_params.values[4][1])         #元胞z方向长度=20
 
stepAvg = int(df_params.values[5][1])              #抽样平均数
stepEquil = float(df_params.values[6][1])          #平衡步长
stepLimit = float(df_params.values[7][1])          #限制的步数
T = float(df_params.values[8][1])        #体系温度

#定义了一个Mol类对象的数组，大小为20*20*20=8000. mol[0]~mol[7999]，共7999个氩原子分子
mol = [Mol(np.asarray([0.0, 0.0, 0.0]), \
           np.asarray([0.0, 0.0, 0.0]), \
           np.asarray([0.0, 0.0, 0.0])) for i in range(int(initUcell[0]*initUcell[1]*initUcell[2]))]

global nMol        #定义分子的数量变量
nMol = len(mol)    #为mol数组的长度为8000

# 氩-氩相互作用的LJ势参数:
epsilon =  1
sigma = 1

# 执行体系初始化相关功能函数
SetParams()                                   #设置主程序内部模拟所需参数
SetupJob()                                    #初始化粒子的坐标，速度，加速
moreCycles = 1                                #MD循环控制开关变量1：run; 0:stop

n = 0                                        #记录步数，用来给每一步输出的坐标图命名                
while moreCycles:                            #MD循环
    SingleStep()                             #求解每一步的运动方程
    if mov==1:
        plotMolLjp(mol, workdir, n)          #制作单个步长的粒子坐标图
    n += 1
    if stepCount >= stepLimit:
        moreCycles = 0
        
#输出模拟过程中生成的参数
columns = ['timestep','timeNow', '$\Sigma v$', 'E', '$\sigma E$', 'Ek', '$\sigma Ek$', 'P_1', 'P_2']
df_systemParams = pd.DataFrame(systemParams, columns=columns)   

#绘制表格，仅在jupyter notebook使用，打印出数据列表
display(df_systemParams) 

if mov==1:                     
    makeMov()                 # 制作视频
GraphOutput()                 #输出体系测定量随模拟时间变化的图像

# END